In [1]:
import torchvision
from torch.utils.tensorboard import SummaryWriter

from model import *
# 准备数据集
from torch import nn
from torch.utils.data import DataLoader

In [2]:
train_data = torchvision.datasets.CIFAR10(root="../data", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="../data", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)


100.0%


Extracting ../data\cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [3]:
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
# 如果train_data_size=10, 训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：50000
测试数据集的长度为：10000


In [4]:
# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
# 创建网络模型
tudui = Tudui()

# 损失函数
loss_fn = nn.CrossEntropyLoss()

# 优化器
# learning_rate = 0.01
# 1e-2=1 x (10)^(-2) = 1 /100 = 0.01
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

In [6]:
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加tensorboard
writer = SummaryWriter("../logs_train")


In [8]:
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))

    # 训练步骤开始
    tudui.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    tudui.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(tudui, "tudui_{}.pth".format(i))
    print("模型已保存")

writer.close()


-------第 1 轮训练开始-------
训练次数：7900, Loss: 1.3901692628860474
训练次数：8000, Loss: 1.1249207258224487
训练次数：8100, Loss: 1.0398794412612915
训练次数：8200, Loss: 1.3268537521362305
训练次数：8300, Loss: 1.266767978668213
训练次数：8400, Loss: 1.118181586265564
训练次数：8500, Loss: 1.2216014862060547
训练次数：8600, Loss: 0.988271176815033
整体测试集上的Loss: 185.71870297193527
整体测试集上的正确率: 0.5812000036239624
模型已保存
-------第 2 轮训练开始-------
训练次数：8700, Loss: 1.304955244064331
训练次数：8800, Loss: 1.42502760887146
训练次数：8900, Loss: 1.0922725200653076
训练次数：9000, Loss: 1.1676270961761475
训练次数：9100, Loss: 1.096876859664917
训练次数：9200, Loss: 1.111709713935852
训练次数：9300, Loss: 1.1000206470489502
整体测试集上的Loss: 180.49519008398056
整体测试集上的正确率: 0.5943999886512756
模型已保存
-------第 3 轮训练开始-------
训练次数：9400, Loss: 0.9282611608505249
训练次数：9500, Loss: 1.304259181022644
训练次数：9600, Loss: 1.1291203498840332
训练次数：9700, Loss: 1.1118332147598267
训练次数：9800, Loss: 0.9378360509872437
训练次数：9900, Loss: 1.0842856168746948
训练次数：10000, Loss: 0.9002047777175903
训练次数：1